In [2]:
!pip install bokeh

   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
    --------------------------------------- 0.1/6.9 MB 2.9 MB/s eta 0:00:03
   -- ------------------------------------- 0.3/6.9 MB 3.6 MB/s eta 0:00:02
   --- ------------------------------------ 0.6/6.9 MB 3.8 MB/s eta 0:00:02
   ---- ----------------------------------- 0.8/6.9 MB 4.5 MB/s eta 0:00:02
   ------ --------------------------------- 1.0/6.9 MB 4.7 MB/s eta 0:00:02
   ------- -------------------------------- 1.3/6.9 MB 4.7 MB/s eta 0:00:02
   --------- ------------------------------ 1.6/6.9 MB 5.1 MB/s eta 0:00:02
   ----------- ---------------------------- 1.9/6.9 MB 5.4 MB/s eta 0:00:01
   ------------- -------------------------- 2.3/6.9 MB 5.6 MB/s eta 0:00:01
   --------------- ------------------------ 2.7/6.9 MB 6.0 MB/s eta 0:00:01
   ------------------ --------------------- 3.2/6.9 MB 6.3 MB/s eta 0:00:01
   --------------------- ------------------ 3.6/6.9 MB 6.6 MB/s eta 0:00:01
   ----------------

In [7]:
!pip install -e .

Obtaining file:///C:/Users/marriotta/code/backtesting/notebooks/backtesting_strategies


ERROR: file:///C:/Users/marriotta/code/backtesting/notebooks/backtesting_strategies does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [5]:
import logging
import sys
import datetime as dt
from bokeh.io import output_notebook, show

from backtesting.main import main

from visulisation import Visulisation
from app import BackTesterApp

# initialise support objects
server = "green"
creds = get_creds(file='../auth/credentials.yaml')
env = get_env(creds['environment'])
fs = env.get_s3fs(server=server, key=creds['minio']['key'], secret=creds['minio']['secret'])
dl = env.get_dataloader('exchange', fs)
vis = Visulisation()

# set logging level
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger("Run")

output_notebook()

ModuleNotFoundError: No module named 'backtesting'

#### load previous snapshot to get active orderbooks

In [2]:
snapshot_date = (dt.datetime.now()- dt.timedelta(days=2)).strftime('%Y-%m-%d')
snapshot = get_liquidity_profile_snapshot(dl, 'ldprof',snapshot_date, snapshot_date)

In [3]:
app = BackTesterApp(snapshot)

## RISK BACKTESTER

In [4]:
app.display()

AppLayout(children=(Label(value='RISK STRATEGY BACKTESTING: for guidance on running backtest please see the RE…

In [5]:
args = app.get_args(app.w, server)

INFO:getArgs:pipeline config created via gui
INFO:getArgs:simulations config created via gui
INFO:getArgs:output config created via gui


In [6]:
%%time
result = main(**args)

INFO:BackTestingConfigBuilder:building auth
INFO:BackTestingConfigBuilder:build config
INFO:SimulationPool:run_simulation_for_batch: (plans) 1, (attempt) 1/3
INFO:SimulationPlan[simulation/81779b03fe9a700361c10039a5764dbd]:Success on plan simulation / 81779b03fe9a700361c10039a5764dbd, start: 2020-12-09, end: 2020-12-11, instruments: [100643], has_events: True, start_time: 2020-12-17 12:15:47.348477, end_time: 2020-12-17 12:17:40.950187, duration: 0:01:53.601710
CPU times: user 233 ms, sys: 269 ms, total: 502 ms
Wall time: 1min 54s


In [12]:
if result.errors == []:
    logger.info('backtest executed with no errors')
else:
    logger.warning('backtest executed with errors, do view the plans and corresponding error, analyse `result.error`')

INFO:Run:backtest executed with no errors


## Visulise Simulation

In [7]:
simulations = result.df

In [8]:
show(vis.plot_sim_profile(simulations[simulations.portfolio=='lmax'], 'rpnl_cum', list(simulations.hash.unique()), resample_rule=None))